In this work, the “Simple Transformers” library is used: https://simpletransformers.ai/.

This library is built on top of huggingface/transformers, and offers models for specific NPL tasks. A model contains the pre-trainded model form Hugging Face and the other additional configurations to fine-tune the pre-trained model for a specific NPL model. We only have to train it.

---
Steps:
1. Initialize the model from the “Simple Transformers” library.

2. Train the model.

3. Test/evaluate the model.

4. Answer questions with the model.



In [ ]:
# Save your model in Google Drive
%cd /content/drive/MyDrive/IIP1/Assignments/Second/ModelFull/acc

/content/drive/MyDrive/IIP1/Assignments/Second/ModelFull/acc


In [ ]:
# Installing the "Simple Transformer" library
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 25.1 MB/s eta 0:00:00

In [ ]:
# Importing the necessary libraries
import logging
import sklearn
import os
import requests
import json

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
# Getting training and testing datasets

os.mkdir('squad')
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'

for file in ['train-v2.0.json', 'dev-v2.0.json']:
  res = requests.get(f'{url}{file}') #make request
  with open(f'squad/{file}', 'wb') as f:
    for chunk in res.iter_content(chunk_size=4):
      f.write(chunk)

In [ ]:
# Preparing a training dataset (130319 samples)

with open("squad/train-v2.0.json", "rb") as f:
  squad_dict = json.load(f)

train = []
for group in squad_dict['data']:
  train = train + group['paragraphs']

In [ ]:
# Preparing an evaluation dataset (used during the training phase) (5448 samples)
with open("squad/dev-v2.0.json", "rb") as f:
  squad_dict = json.load(f)

eval = []
for i in range(17):
  eval = eval + squad_dict['data'][i]['paragraphs']

In [ ]:
# Preparing a testing dataset (6425 samples)
with open("squad/dev-v2.0.json", "rb") as f:
  squad_dict = json.load(f)

test = []
for i in range(17,35):
  test = test + squad_dict['data'][i]['paragraphs']

In [ ]:
#Checking the required format for each dataset, based on:
# https://simpletransformers.ai/docs/qa-data-formats/

print(train[0])
print(eval[0])
print(test[0])

{'qas': [{'question': 'When did Beyonce start becoming popular?', 'id': '56be85543aeaaa14008c9063', 'answers': [{'text': 'in the late 1990s', 'answer_start': 269}], 'is_impossible': False}, {'question': 'What areas did Beyonce compete in when she was growing up?', 'id': '56be85543aeaaa14008c9065', 'answers': [{'text': 'singing and dancing', 'answer_start': 207}], 'is_impossible': False}, {'question': "When did Beyonce leave Destiny's Child and become a solo singer?", 'id': '56be85543aeaaa14008c9066', 'answers': [{'text': '2003', 'answer_start': 526}], 'is_impossible': False}, {'question': 'In what city and state did Beyonce  grow up? ', 'id': '56bf6b0f3aeaaa14008c9601', 'answers': [{'text': 'Houston, Texas', 'answer_start': 166}], 'is_impossible': False}, {'question': 'In which decade did Beyonce become famous?', 'id': '56bf6b0f3aeaaa14008c9602', 'answers': [{'text': 'late 1990s', 'answer_start': 276}], 'is_impossible': False}, {'question': 'In what R&B group was she the lead singer?',

---

---
1. **Initialize the model from the “Simple Transformers” library.**

    At first, we have to determine the model type and the pre-trained model type. Then, we have to set the training arguments for our model.

In [ ]:
# Here BERT model is used for the "Question Answering" task.
# The pre-trained model is 'bert-base-cased' from Hugging Face: https://huggingface.co/bert-base-cased

model_type= 'bert'
model_name= 'bert-base-cased'

In [ ]:
# https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model

train_args = {
    #general arguments
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    # Maximum sequence length the model will support.
    "max_seq_length": 128,
    # The number of predictions given per question.
    "n_best_size":3,
    "train_batch_size": 128,
    "num_train_epochs": 50,
    "evaluate_during_training": True,
    "use_cached_eval_features": True,
    "eval_batch_size": 64,
    # Evaluate the model for each 800 optimization steps
    "evaluate_during_training_steps": 800,

    # Arguments for early stopping
    "use_early_stopping": True,
    "early_stopping_metric": "em_score",
    "early_stopping_metric_minimize": False, # To maximize "em_score"
    "early_stopping_delta": 0.01,  # Stop if "em_score" not improved with 0.01
    "early_stopping_patience": 2,  # for 3 consecutive evaluations (detect + 2).


     # Tracking the training process with Weights & Biases AI: https://wandb.ai/
    "wandb_project": "Question Answer Models",
    "wandb_kwargs": {"name": "BERT Question Answering"},
}

In [ ]:
# Initializing the model
model = QuestionAnsweringModel(
    model_type, model_name, args=train_args
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---





---
2. **Train the model.**

    In case of not an early stop, the model will be trained for 50 epochs, with 1029 steps per epoch. (step = one pass forward and backward)



---------------------------


    *In output section, train_acc refers to em_score

In [ ]:
# Train the model
model.train_model(train, eval_data=eval, em_score=sklearn.metrics.accuracy_score)

add example index and unique id: 100%|██████████| 130319/130319 [00:00<00:00, 812436.46it/s]


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Running Evaluation:   0%|          | 0/89 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/89 [00:00<?, ?it/s]

Running Epoch 6 of 50:   0%|          | 0/1029 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/89 [00:00<?, ?it/s]

(6400,
 {'global_step': [800,
   1029,
   1600,
   2058,
   2400,
   3087,
   3200,
   4000,
   4116,
   4800,
   5145,
   5600,
   6174,
   6400],
  'correct': [2375,
   2556,
   2998,
   2811,
   2888,
   2973,
   3136,
   3217,
   3193,
   3147,
   3160,
   3044,
   3033,
   3169],
  'similar': [2763,
   2632,
   2276,
   2411,
   2323,
   2271,
   2139,
   2090,
   2089,
   2141,
   2128,
   2207,
   2187,
   2108],
  'incorrect': [310,
   260,
   174,
   226,
   237,
   204,
   173,
   141,
   166,
   160,
   160,
   197,
   228,
   171],
  'train_loss': [1.2567840814590454,
   0.9651287198066711,
   0.8753485679626465,
   0.8042511940002441,
   0.8067566156387329,
   0.8875826597213745,
   0.6612769365310669,
   0.7724835872650146,
   0.6025730967521667,
   0.49927055835723877,
   0.28688037395477295,
   0.41746005415916443,
   0.35443243384361267,
   0.19979792833328247],
  'eval_loss': [-2.9169373244382024,
   -3.629476825842697,
   -5.041915379213483,
   -4.977923103932584,
  

---